In [182]:
#!/usr/local/bin/python
# -*- coding: utf-8 -*-
# Copyright (c) 2017 Mikhail Shilov

import numpy as np
import pandas as pd
import plotly.plotly as py
from plotly import graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected = True)

path = "stat_dash.csv"  # данные взяты тут https://explorer.dash.org/chain/Dash/q/nethash
data = pd. read_csv(path, delimiter=";")
data['time'] =  pd.to_datetime(data.time, unit='s')


data = data[['time', 'netHashPerSecond']]
data.set_index(data['time'],inplace=True)
data.head()

,time,netHashPerSecond
time,,
2014-01-19 04:00:14,2014-01-19 04:00:14,17984
2014-01-19 04:03:56,2014-01-19 04:03:56,680168
2014-01-19 04:07:15,2014-01-19 04:07:15,758780
2014-01-19 04:10:50,2014-01-19 04:10:50,702313
2014-01-19 04:19:40,2014-01-19 04:19:40,1139602


In [180]:
#импортируем библиотеку
from fbprophet import Prophet
predictions = 522   # кол-во прогнозируемых дней

# приводим dataframe к нужному формату
df = data.reset_index(drop=True)
df.columns = [ 'ds', 'y']
df.set_index(df['ds'],inplace=True)
#df['y'] = np.log10(df['y'])
m = Prophet()
m.fit(df)

future = m.make_future_dataframe(periods=predictions)
forecast = m.predict(future)

cmp_df = forecast.set_index('ds')[['yhat', 'yhat_lower', 'yhat_upper']].join(df.set_index('ds'))


In [183]:
# функция для визуализации построенного прогноза
def show_forecast(cmp_df):

    # прогноз
    forecast = go.Scatter(
        name='Prediction',
        x=cmp_df.tail(predictions).index,
        y=cmp_df.tail(predictions).yhat,
        mode='lines',
        line=dict(color='rgb(31, 119, 180)'),
    )
  
    # фактические значения
    fact = go.Scatter(
        name='Fact',
        x=cmp_df.index,
        y=cmp_df.y,
        marker=dict(color="red"),
        mode='lines',
    )

    # последовательность рядов в данном случае важна из-за применения заливки
    data = [ forecast, fact]

    layout = go.Layout(
        yaxis=dict(title='Хэшрейт'),
        title='Прогнозируемый хэшрейт сети DASH',
        showlegend = False)

    fig = go.Figure(data=data, layout=layout)
    iplot(fig, show_link=False)

show_forecast(cmp_df)